In [ ]:
from dolfin import *
import numpy as np
from numpy.random import random


# Sub domain for Periodic boundary condition
class PeriodicBoundary(SubDomain):

    # Left boundary is "target domain" G
    def inside(self, x, on_boundary):
        # return True if on left or bottom boundary AND NOT on one of the two corners (0, 1) and (1, 0)
        return bool((near(x[0], 0) or near(x[1], 0)) and \
                (not ((near(x[0], 0) and near(x[1], 1)) or \
                        (near(x[0], 1) and near(x[1], 0)))) and on_boundary)





    # Map right boundary (H) to left boundary (G)
    def map(self, x, y):
        if near(x[0], 1) and near(x[1], 1):
            y[0] = x[0] - 1.
            y[1] = x[1] - 1.
        elif near(x[0], 1):
            y[0] = x[0] - 1.
            y[1] = x[1]
        else:   # near(x[1], 1)
            y[0] = x[0]
            y[1] = x[1] - 1.

#Initial condtions (Random)
class IC(UserExpression):
    def eval(self,values,x):
        values[0] = 0.1*random() -0.1*random()
        values[1] = 0.1*random() -0.1*random()
    def value_shape(self):
        return(2,)




#Parameters
dt = 2.0e-02

a=Constant(1.0)
b=Constant(-1.0)
c=Constant(2)
d=Constant(-1.5)
mu=Constant(0.0001)
nu=Constant(0.0006)

# Form compiler options
parameters["form_compiler"]["optimize"]     = True
parameters["form_compiler"]["cpp_optimize"] = True
#parameters["form_compiler"]["representation"] = "quadrature"

# Create mesh and finite element
mesh = UnitSquareMesh(60, 60 )
mesh=refine(mesh)
P1 = FiniteElement("P", triangle, 1)
element = MixedElement([P1, P1])
V = FunctionSpace(mesh, element, constrained_domain=PeriodicBoundary())
plot(mesh)

# Condensed definition and encapsulation of functions
du       =TrialFunction(V)
v_1, v_2 = TestFunctions(V)
u = Function(V)
u_n = Function(V)

# Split system functions to access components
dc,dmu=split(du)
u_1, u_2= split(u)
u_n1, u_n2 = split(u_n)

#Initializzatioin
u1init = IC(element = u.ufl_element())
u.interpolate(u1init)
u_n.interpolate(u1init)


# Define variational problem

F  = (u_1-u_n1)*v_1*dx \
     +mu*dt*inner(grad(u_1),grad(v_1))*dx\
     -dt*(a*u_1+b*u_2)*v_1*dx\
     +(u_2-u_n2)*v_2*dx\
     +nu*dt*inner(grad(u_2),grad(v_2))*dx\
     -dt*(c*u_1+d*u_2)*v_2*dx

  
#Time parameters:
t = 0.0
T=900*dt
#vtk file
vtkfile_u_1 = File("Turing_new/u_1.pvd","compressed")
vtkfile_u_2 = File("Turing_new/u_2.pvd","compressed")

#Jacobian, gateau derivative direction (du)
J=derivative(F,u,du)
params = {'nonlinear_solver': 'snes',
           'snes_solver':
            {
                'linear_solver'           : 'umfpack',
                'absolute_tolerance'      : 1e-10,
                'relative_tolerance'      : 1e-10,
                'maximum_iterations'      : 20,
             }
         }



while (t<T):
# Update current time
    t += dt
    u_n.assign(u)
    solve(F==0,u,J=J,solver_parameters=params)
# Save solution to file (VTK)
    _u_1, _u_2 = u.split()
    vtkfile_u_1 << (_u_1, t)
    vtkfile_u_2 << (_u_2, t)
# Update previous solution
    
    plot(u_1)

